In [1]:
import os
import pandas as pd 

In [38]:
dd = './gambling_asesores_final/'
assessments = []
for f in os.listdir(dd):
    df = pd.read_csv(dd+'/'+f, index_col=0)
    df["topic"] = int(f.replace('.csv','')) + 1
    assessments.append(df)
    print("Number of relevants per consensus", df[df["consensus"]==True]['doc_id'].count(), "out of", len(df))
    print("Number of relevants per majority", df[df["majority"]==True]['doc_id'].count(), "out of", len(df))
    print()

Number of relevants per consensus 13 out of 134
Number of relevants per majority 27 out of 134

Number of relevants per consensus 3 out of 121
Number of relevants per majority 12 out of 121

Number of relevants per consensus 14 out of 114
Number of relevants per majority 23 out of 114

Number of relevants per consensus 11 out of 103
Number of relevants per majority 24 out of 103

Number of relevants per consensus 3 out of 117
Number of relevants per majority 11 out of 117

Number of relevants per consensus 3 out of 109
Number of relevants per majority 8 out of 109

Number of relevants per consensus 8 out of 107
Number of relevants per majority 16 out of 107

Number of relevants per consensus 3 out of 103
Number of relevants per majority 7 out of 103

Number of relevants per consensus 8 out of 91
Number of relevants per majority 16 out of 91



In [39]:
final = pd.concat(assessments)
print(final.columns)

Index(['doc_id', 'text', 'manu.couto1k_rels', 'marcosfp_rels',
       'ezra.mario_rels', 'consensus', 'majority', 'topic'],
      dtype='object')


In [40]:
final

,doc_id,text,manu.couto1k_rels,marcosfp_rels,ezra.mario_rels,consensus,majority,topic
0,98560,La ludopatía tiene buen pronóstico en régimen ...,False,False,False,False,False,6
1,442115,Y tú poder demostrarle a ella a quien tanto he...,True,True,False,False,True,6
2,418052,11 Ha ocultado alguna vez a su pareja a sus hi...,True,False,False,False,False,6
3,66309,Deberías de plantearte si tu situación antisoc...,False,False,False,False,False,6
4,20486,Es posible que hayas dado este paso agobiado p...,False,False,False,False,False,6
...,...,...,...,...,...,...,...,...
86,294647,es que no puedo jugar joder no puedo jugar yo ...,False,False,False,False,False,7
87,471288,Ha intentado usted alguna vez reducir su juego,True,False,False,False,False,7
88,75513,Evidentemente el que no quiere dejar de jugar ...,False,False,False,False,False,7
89,447739,Ha intentado dejar de jugar o jugar menos din...,True,False,False,False,False,7


In [70]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(final['manu.couto1k_rels'].values, final['marcosfp_rels'].values)

0.4621155767146289

In [71]:
cohen_kappa_score(final['manu.couto1k_rels'], final['ezra.mario_rels'])

0.41305269904706243

In [72]:
cohen_kappa_score(final['marcosfp_rels'], final['ezra.mario_rels'])

0.44795325705143807

In [73]:
import krippendorff
krippendorff.alpha(reliability_data=[list(final['marcosfp_rels'].astype(int).values), list(final['ezra.mario_rels'].astype(int).values),list(final['manu.couto1k_rels'].astype(int).values)])

0.43685967429961936

In [74]:
#### Automatic labelling GPT-4
import numpy as np 

gpt4 = pd.read_csv('./outputs/gpt-4.csv', names=["topic","doc_id", "label"])
gpt4['label'] = np.where(gpt4['label']=="Relevante", True, gpt4['label'])
gpt4['label'] = np.where(gpt4['label']=="No relevante", False, gpt4['label'])
gpt4

,topic,doc_id,label
0,1,265216,True
1,1,92161,False
2,1,68608,False
3,1,483331,True
4,1,130052,True
...,...,...,...
3160,9,357360,False
3161,9,10226,True
3162,9,342004,True
3163,9,10231,False


In [75]:
final.columns

Index(['doc_id', 'text', 'manu.couto1k_rels', 'marcosfp_rels',
       'ezra.mario_rels', 'consensus', 'majority', 'topic'],
      dtype='object')

In [76]:
gpt4.columns

Index(['topic', 'doc_id', 'label'], dtype='object')

In [77]:
final['doc_id'] = final['doc_id'].astype(int)
final['topic'] = final['topic'].astype(int)
gpt4['doc_id'] = gpt4['doc_id'].astype(int)
gpt4['topic'] = gpt4['topic'].astype(int)
final = final.merge(gpt4, how='left', on=["doc_id", "topic"])
final["label"] = final["label"].astype(bool)
final

,doc_id,text,manu.couto1k_rels,marcosfp_rels,ezra.mario_rels,consensus,majority,topic,label
0,98560,La ludopatía tiene buen pronóstico en régimen ...,False,False,False,False,False,6,False
1,442115,Y tú poder demostrarle a ella a quien tanto he...,True,True,False,False,True,6,True
2,418052,11 Ha ocultado alguna vez a su pareja a sus hi...,True,False,False,False,False,6,True
3,66309,Deberías de plantearte si tu situación antisoc...,False,False,False,False,False,6,False
4,20486,Es posible que hayas dado este paso agobiado p...,False,False,False,False,False,6,False
...,...,...,...,...,...,...,...,...,...
994,294647,es que no puedo jugar joder no puedo jugar yo ...,False,False,False,False,False,7,True
995,471288,Ha intentado usted alguna vez reducir su juego,True,False,False,False,False,7,True
996,75513,Evidentemente el que no quiere dejar de jugar ...,False,False,False,False,False,7,False
997,447739,Ha intentado dejar de jugar o jugar menos din...,True,False,False,False,False,7,True


In [78]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(final['label'].values, final['ezra.mario_rels'].values)

0.3247142676675333

In [79]:
cohen_kappa_score(final['label'].values, final['marcosfp_rels'].values)

0.22648608998043906

In [80]:
cohen_kappa_score(final['label'].values, final['manu.couto1k_rels'].values)

0.42149202658536045

In [81]:
#### KAPPA CON LOS QRELS DE CONSENSUS

cohen_kappa_score(final['label'].values, final['consensus'].values)

0.14741824321519026

In [82]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(final['consensus'].values, final['label'].values).ravel()
print(tn, fp, fn, tp)

569 364 6 60


In [83]:
#### KAPPA CON LOS QRELS DE MAJORITY

cohen_kappa_score(final['label'].values, final['majority'].values)

0.29559417946645106

In [84]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(final['majority'].values, final['label'].values).ravel()
print(tn, fp, fn, tp)

558 297 17 127


In [35]:
#### SEGUNDO ETIQUETADO CON GPT-4

import numpy as np 

gpt42 = pd.read_csv('./outputs/gpt-4-2.csv', names=["topic","doc_id", "label"])
gpt42['label'] = np.where(gpt42['label']=="Relevante", True, gpt42['label'])
gpt42['label'] = np.where(gpt42['label']=="No relevante", False, gpt42['label'])
gpt42

,topic,doc_id,label
0,1,265216,True
1,1,92161,False
2,1,68608,False
3,1,483331,True
4,1,130052,True
...,...,...,...
3159,9,357360,False
3160,9,10226,True
3161,9,342004,True
3162,9,10231,False


In [90]:
final['doc_id'] = final['doc_id'].astype(int)
final['topic'] = final['topic'].astype(int)
gpt42['doc_id'] = gpt42['doc_id'].astype(int)
gpt42['topic'] = gpt42['topic'].astype(int)
final = final.merge(gpt42, how='left', on=["doc_id", "topic"])
final["label"] = final["label"].astype(bool)
final

,doc_id,text,manu.couto1k_rels,marcosfp_rels,ezra.mario_rels,consensus,majority,topic,label
0,98560,La ludopatía tiene buen pronóstico en régimen ...,False,False,False,False,False,6,False
1,442115,Y tú poder demostrarle a ella a quien tanto he...,True,True,False,False,True,6,True
2,418052,11 Ha ocultado alguna vez a su pareja a sus hi...,True,False,False,False,False,6,True
3,66309,Deberías de plantearte si tu situación antisoc...,False,False,False,False,False,6,False
4,20486,Es posible que hayas dado este paso agobiado p...,False,False,False,False,False,6,False
...,...,...,...,...,...,...,...,...,...
994,294647,es que no puedo jugar joder no puedo jugar yo ...,False,False,False,False,False,7,True
995,471288,Ha intentado usted alguna vez reducir su juego,True,False,False,False,False,7,False
996,75513,Evidentemente el que no quiere dejar de jugar ...,False,False,False,False,False,7,False
997,447739,Ha intentado dejar de jugar o jugar menos din...,True,False,False,False,False,7,True


In [91]:
#### KAPPA CON LOS QRELS DE CONSENSUS

cohen_kappa_score(final['label'].values, final['consensus'].values)

0.11283979378827558

In [92]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(final['consensus'].values, final['label'].values).ravel()
print(tn, fp, fn, tp)

473 460 2 64


In [62]:
#### KAPPA CON LOS QRELS DE MAJORITY

cohen_kappa_score(final['label'].values, final['majority'].values)

0.23394911118856743

In [64]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(final['majority'].values, final['label'].values).ravel()
print(tn, fp, fn, tp)

467 388 8 136


In [12]:
###### ETIQUETADO CON GEMINI

import numpy as np 

gem = pd.read_csv('./outputs/gemini.csv', names=["topic","doc_id", "label"])
gem['label'] = np.where(gem['label']=="Relevante", True, gem['label'])
gem['label'] = np.where(gem['label']=="No relevante", False, gem['label'])
gem = gem[gem["topic"]==1]

In [13]:
gem1 = pd.read_csv('./outputs/gemini-2.csv', names=["topic","doc_id", "label"])
gem1['label'] = np.where(gem1['label']=="Relevante", True, gem1['label'])
gem1['label'] = np.where(gem1['label']=="No relevante", False, gem1['label'])

In [14]:
gem3 = pd.read_csv('./outputs/gemini-3.csv', names=["topic","doc_id", "label"])
gem3['label'] = np.where(gem3['label']=="Relevante", True, gem3['label'])
gem3['label'] = np.where(gem3['label']=="No relevante", False, gem3['label'])
gem3 = gem3[gem3["topic"]==7]

In [15]:
gem4 = pd.read_csv('./outputs/gemini-4.csv', names=["topic","doc_id", "label"])
gem4['label'] = np.where(gem4['label']=="Relevante", True, gem4['label'])
gem4['label'] = np.where(gem4['label']=="No relevante", False, gem4['label'])

In [16]:
gem = pd.concat([gem, gem1, gem3, gem4])
gem

,topic,doc_id,label
0,1,265216,False
1,1,92161,False
2,1,68608,False
3,1,483331,True
4,1,130052,True
...,...,...,...
721,9,357360,False
722,9,10226,True
723,9,342004,True
724,9,10231,False


In [24]:
gem.loc[gem['label'].str.contains('Relevante', na=False), 'label'] = True
gem.loc[gem['label'].str.contains('No relevante', na=False), 'label'] = False
gem.loc[gem['label'].str.contains('Irrelevante', na=False), 'label'] = False
gem.loc[gem['label'].str.contains('No es relevante', na=False), 'label'] = False
gem.loc[gem['label'].str.contains('No pude generar contenido', na=False), 'label'] = False
gem.loc[gem['label'].str.contains('apostar con más y más', na=False), 'label'] = False

In [26]:
gem["label"].value_counts()
gem.to_csv('./outputs/full_gemini.csv', header=False, index=False)

In [41]:
final['doc_id'] = final['doc_id'].astype(int)
final['topic'] = final['topic'].astype(int)
gem['doc_id'] = gem['doc_id'].astype(int)
gem['topic'] = gem['topic'].astype(int)
final = final.merge(gem, how='left', on=["doc_id", "topic"])
final["label"] = final["label"].astype(bool)
final

,doc_id,text,manu.couto1k_rels,marcosfp_rels,ezra.mario_rels,consensus,majority,topic,label
0,98560,La ludopatía tiene buen pronóstico en régimen ...,False,False,False,False,False,6,True
1,442115,Y tú poder demostrarle a ella a quien tanto he...,True,True,False,False,True,6,True
2,418052,11 Ha ocultado alguna vez a su pareja a sus hi...,True,False,False,False,False,6,True
3,66309,Deberías de plantearte si tu situación antisoc...,False,False,False,False,False,6,True
4,20486,Es posible que hayas dado este paso agobiado p...,False,False,False,False,False,6,False
...,...,...,...,...,...,...,...,...,...
994,294647,es que no puedo jugar joder no puedo jugar yo ...,False,False,False,False,False,7,True
995,471288,Ha intentado usted alguna vez reducir su juego,True,False,False,False,False,7,False
996,75513,Evidentemente el que no quiere dejar de jugar ...,False,False,False,False,False,7,False
997,447739,Ha intentado dejar de jugar o jugar menos din...,True,False,False,False,False,7,True


In [31]:
from sklearn.metrics import cohen_kappa_score

cohen_kappa_score(final['label'].values, final['consensus'].values)

0.09783590290395916

In [33]:
cohen_kappa_score(final['label'].values, final['majority'].values)

0.15953772582359182

In [32]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(final['consensus'].values, final['label'].values).ravel()
print(tn, fp, fn, tp)

437 496 2 64


#### VOTING DE GPT-4 Y GEMINI

In [34]:
final

,doc_id,text,manu.couto1k_rels,marcosfp_rels,ezra.mario_rels,consensus,majority,topic,label
0,98560,La ludopatía tiene buen pronóstico en régimen ...,False,False,False,False,False,6,True
1,442115,Y tú poder demostrarle a ella a quien tanto he...,True,True,False,False,True,6,True
2,418052,11 Ha ocultado alguna vez a su pareja a sus hi...,True,False,False,False,False,6,True
3,66309,Deberías de plantearte si tu situación antisoc...,False,False,False,False,False,6,True
4,20486,Es posible que hayas dado este paso agobiado p...,False,False,False,False,False,6,False
...,...,...,...,...,...,...,...,...,...
994,294647,es que no puedo jugar joder no puedo jugar yo ...,False,False,False,False,False,7,True
995,471288,Ha intentado usted alguna vez reducir su juego,True,False,False,False,False,7,False
996,75513,Evidentemente el que no quiere dejar de jugar ...,False,False,False,False,False,7,False
997,447739,Ha intentado dejar de jugar o jugar menos din...,True,False,False,False,False,7,True


In [43]:
final['doc_id'] = final['doc_id'].astype(int)
final['topic'] = final['topic'].astype(int)
gpt42['doc_id'] = gpt42['doc_id'].astype(int)
gpt42['topic'] = gpt42['topic'].astype(int)
final = final.merge(gpt42, how='left', on=["doc_id", "topic"])

,doc_id,text,manu.couto1k_rels,marcosfp_rels,ezra.mario_rels,consensus,majority,topic,label_gem,label_gpt4,label
0,98560,La ludopatía tiene buen pronóstico en régimen ...,False,False,False,False,False,6,True,False,False
1,442115,Y tú poder demostrarle a ella a quien tanto he...,True,True,False,False,True,6,True,True,True
2,418052,11 Ha ocultado alguna vez a su pareja a sus hi...,True,False,False,False,False,6,True,True,True
3,66309,Deberías de plantearte si tu situación antisoc...,False,False,False,False,False,6,True,False,False
4,20486,Es posible que hayas dado este paso agobiado p...,False,False,False,False,False,6,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
994,294647,es que no puedo jugar joder no puedo jugar yo ...,False,False,False,False,False,7,True,True,True
995,471288,Ha intentado usted alguna vez reducir su juego,True,False,False,False,False,7,False,False,False
996,75513,Evidentemente el que no quiere dejar de jugar ...,False,False,False,False,False,7,False,False,False
997,447739,Ha intentado dejar de jugar o jugar menos din...,True,False,False,False,False,7,True,True,True


In [49]:
final.rename(columns={'label_x':'label_gem', 'label_y':'label_gpt4'}, inplace=True)

In [50]:
final["label"] = final["label_gem"] & final["label_gpt4"]
final

,doc_id,text,manu.couto1k_rels,marcosfp_rels,ezra.mario_rels,consensus,majority,topic,label_gem,label_gpt4,label
0,98560,La ludopatía tiene buen pronóstico en régimen ...,False,False,False,False,False,6,True,False,False
1,442115,Y tú poder demostrarle a ella a quien tanto he...,True,True,False,False,True,6,True,True,True
2,418052,11 Ha ocultado alguna vez a su pareja a sus hi...,True,False,False,False,False,6,True,True,True
3,66309,Deberías de plantearte si tu situación antisoc...,False,False,False,False,False,6,True,False,False
4,20486,Es posible que hayas dado este paso agobiado p...,False,False,False,False,False,6,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
994,294647,es que no puedo jugar joder no puedo jugar yo ...,False,False,False,False,False,7,True,True,True
995,471288,Ha intentado usted alguna vez reducir su juego,True,False,False,False,False,7,False,False,False
996,75513,Evidentemente el que no quiere dejar de jugar ...,False,False,False,False,False,7,False,False,False
997,447739,Ha intentado dejar de jugar o jugar menos din...,True,False,False,False,False,7,True,True,True


In [51]:
from sklearn.metrics import cohen_kappa_score

cohen_kappa_score(final['label'].values, final['consensus'].values)

0.15601886701242373

In [52]:
cohen_kappa_score(final['label'].values, final['majority'].values)

0.26312072479267135

In [53]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(final['consensus'].values, final['label'].values).ravel()
print(tn, fp, fn, tp)

570 363 4 62


##### ETIQUETADO PRE-GROUP MEETING

In [5]:
dd = './gambling_asesores_primero/'
assessments = []
for f in os.listdir(dd):
    df = pd.read_csv(dd+'/'+f, index_col=0)
    df = df[['doc_id', 'text', 'manu.couto1k_rels', 'marcosfp_rels', 'ezra_rels', 'consensus', 'majority']]
    assessments.append(df)
    print("Number of relevants per consensus", df[df["consensus"]==True]['doc_id'].count(), "out of", len(df))
    print("Number of relevants per majority", df[df["majority"]==True]['doc_id'].count(), "out of", len(df))
    print()

Number of relevants per consensus 2 out of 60
Number of relevants per majority 8 out of 60

Number of relevants per consensus 11 out of 72
Number of relevants per majority 13 out of 72

Number of relevants per consensus 4 out of 77
Number of relevants per majority 15 out of 77

Number of relevants per consensus 2 out of 74
Number of relevants per majority 10 out of 74

Number of relevants per consensus 4 out of 67
Number of relevants per majority 9 out of 67

Number of relevants per consensus 7 out of 62
Number of relevants per majority 16 out of 62

Number of relevants per consensus 0 out of 71
Number of relevants per majority 5 out of 71

Number of relevants per consensus 2 out of 63
Number of relevants per majority 9 out of 63

Number of relevants per consensus 3 out of 67
Number of relevants per majority 7 out of 67



In [6]:
final = pd.concat(assessments)
print(final.columns)

Index(['doc_id', 'text', 'manu.couto1k_rels', 'marcosfp_rels', 'ezra_rels',
       'consensus', 'majority'],
      dtype='object')


In [7]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(final['manu.couto1k_rels'].values, final['marcosfp_rels'].values)

0.38692643117174064

In [9]:
cohen_kappa_score(final['manu.couto1k_rels'], final['ezra_rels'])

0.3450240875775582

In [11]:
cohen_kappa_score(final['marcosfp_rels'], final['ezra_rels'])

0.434204872770095

In [13]:
import krippendorff
krippendorff.alpha(reliability_data=[list(final['marcosfp_rels'].astype(int).values), list(final['ezra_rels'].astype(int).values),list(final['manu.couto1k_rels'].astype(int).values)])

0.38661958157371